In [1]:
import os
import sys
import json

#os.environ['CUDA_VISIBLE_DEVICES']='1'
sys.path.append('..')

import config

In [2]:
from utils.dataset import ImageNetDataset
from utils.transform import train_transform, valid_transform
from utils.generator import generator
from utils.callbacks import get_callbacks

Using TensorFlow backend.


# Prepare data

In [3]:
# labels mapping
with open(config.MAP_CLS) as f:
    label_to_class = json.load(f)  
folder_to_label = {v[0]: k for k, v in label_to_class.items()}

In [4]:
from albumentations import (
    HorizontalFlip,  ShiftScaleRotate, RGBShift, CenterCrop,
    RandomSizedCrop, SmallestMaxSize, RandomCrop,
    ShiftScaleRotate, HueSaturationValue, Normalize,
    RandomContrast, RandomBrightness, Flip, OneOf, Compose
)

train_aug = Compose([
    SmallestMaxSize(max_size=256),
    ShiftScaleRotate(scale_limit=(0.5, 1), rotate_limit=5),
    RandomCrop(224, 224, p=1.0),
    HorizontalFlip(0.5),
    OneOf([
        HueSaturationValue(hue_shift_limit=5, sat_shift_limit=5, val_shift_limit=5, p=1),
        RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=0.5)
    ], p=0.2),
    OneOf([
        RandomBrightness(limit=0.2, p=1.),
        RandomContrast(limit=0.2, p=1.),
    ], p=0.2),
], p = 1.0)


valid_aug = Compose([
    SmallestMaxSize(max_size=256),
    CenterCrop(224, 224, p=1.0),
], p = 1.0)


def make_transform(preprocessing_fn=None, augmenter=None):
    
    def transform(**sample):

        if augmenter is not None:
            sample = augmenter(**sample)

        if preprocessing_fn is not None:
            sample['image'] = preprocessing_fn(sample['image'].astype('float32')).copy()
            
        return sample
    return transform

In [5]:
identity = lambda x: x.copy()

In [6]:
train_transform = make_transform(augmenter=train_aug, preprocessing_fn=identity)
valid_transform = make_transform(augmenter=valid_aug, preprocessing_fn=identity)

In [7]:
# prepare datasets
train_dataset = ImageNetDataset(config.TRAIN_DIR, folder_to_label, transform=train_transform)
valid_dataset = ImageNetDataset(config.VALID_DIR, folder_to_label, transform=valid_transform)

In [30]:
# prepare generators 
train_gen = generator(train_dataset, batch_size=256, num_workers=12, shuffle=True)
valid_gen = generator(valid_dataset, batch_size=16, num_workers=12, shuffle=False)

# Build Model

In [13]:
from nn_models.classification_models import ResNet18, ResNet34, ResNet50, ResNet101, ResNet152, ResNeXt50, ResNeXt101
from nn_models.classification_models import SEResNet18

In [10]:
from keras.optimizers import SGD
from keras.utils import multi_gpu_model

In [14]:
# prepare model

model = SEResNet18((224, 224, 3))
model.load_weights('../../../.keras/models/resnet18_imagenet_1000.h5', by_name=True)

In [20]:
for l in model.layers:
    if l.name.startswith('dense'):
        l.trainable = True
    else:
        l.trainable = False

In [24]:
mg_model = multi_gpu_model(model, gpus=2)

In [34]:
#model.load_weights('../../checkpoints/se_resnet18/weights_ep210.h5')

In [25]:
opt = SGD(lr=0.6, momentum=0.9)
mg_model.compile(opt, 'categorical_crossentropy', 
              ['categorical_accuracy', 'top_k_categorical_accuracy'])

In [26]:
callbacks = get_callbacks('se_resnet18_v2', 
                          checkpoints_dir=config.CHECKPOINTS_DIR, 
                          monitor='val_loss',
                          log_dir=config.LOGS_DIR)

Instructions for updating:
Use the retry module or similar alternatives.


In [15]:
%%time

valid_gen = generator(valid_dataset, batch_size=16, num_workers=12, shuffle=False)

scores = model.evaluate_generator(valid_gen, steps=20) #3125
print('{:.4} {:.4}'.format(scores[1], scores[2]))

0.0 0.0
CPU times: user 3.53 s, sys: 1.43 s, total: 4.96 s
Wall time: 5.35 s


# Train

In [ ]:
mg_model.fit_generator(train_gen,
                    steps_per_epoch=1000, 
                    initial_epoch=0,
                    epochs=10, 
                    validation_data=valid_gen, 
                    validation_steps=1000, 
                    callbacks=callbacks)

Epoch 1/10

Epoch 00001: LearningRateScheduler reducing learning rate to 0.6000000238418579.
1000/1000 [==============================] - 339s 339ms/step - loss: 1.5848 - categorical_accuracy: 0.6269 - top_k_categorical_accuracy: 0.8400 - val_loss: 7.5706 - val_categorical_accuracy: 0.0040 - val_top_k_categorical_accuracy: 0.0163

Epoch 00001: val_loss improved from inf to 7.57059, saving model to /home/user/project/checkpoints/se_resnet18_v2/weights-0.004.h5
Epoch 2/10

Epoch 00002: LearningRateScheduler reducing learning rate to 0.6000000238418579.
1000/1000 [==============================] - 332s 332ms/step - loss: 1.5452 - categorical_accuracy: 0.6339 - top_k_categorical_accuracy: 0.8436 - val_loss: 7.9289 - val_categorical_accuracy: 0.0061 - val_top_k_categorical_accuracy: 0.0226

Epoch 00002: val_loss did not improve
Epoch 3/10

Epoch 00003: LearningRateScheduler reducing learning rate to 0.6000000238418579.
1000/1000 [==============================] - 337s 337ms/step - loss: 1.5

In [23]:
model.save_weights('../../checkpoints/se_resnet18/weights_ep310.h5')